In [202]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_v2_max_72_h.csv')

In [284]:
df2 = df.copy()

In [285]:
creatinine_cols = [c for c in df.columns if 'creatinine' in c.lower()]
UMOL_L_TO_MG_DL = 1 / 88.4

df[creatinine_cols] = df[creatinine_cols] * UMOL_L_TO_MG_DL

glucose_cols = [c for c in df.columns if 'glucose' in c.lower()]
MMOL_L_TO_MG_DL_GLUCOSE = 18.0

df[glucose_cols] = df[glucose_cols] * MMOL_L_TO_MG_DL_GLUCOSE

In [287]:
def normalize_oxygen_saturation(series, name="sao2"):
    """
    Normalize SaO2/SpO2 to percent (0–100).
    Handles mixed fraction (0–1) and percent (21–100).
    Invalid values set to NaN.
    """
    s = series.copy()

    # initialize output
    out = pd.Series(np.nan, index=s.index, dtype="float")

    # fraction values (0–1.1) → percent
    frac_mask = (s >= 0) & (s <= 1.1)
    out.loc[frac_mask] = s.loc[frac_mask] * 100.0

    # percent values (1.1–100)
    pct_mask = (s > 1.1) & (s <= 100)
    out.loc[pct_mask] = s.loc[pct_mask]

    # everything else stays NaN
    return out

sao2_cols = [c for c in df.columns if "sao2" in c.lower()]
for c in sao2_cols:
    df[c] = normalize_oxygen_saturation(df[c], name=c)

sp2_cols = [c for c in df.columns if "spo2" in c.lower()]
for c in sao2_cols:
    df[c] = normalize_oxygen_saturation(df[c], name=c)

df["fio2_mean"] = (df["fio2_mean"]/100)

In [289]:
def normalize_temperature_to_celsius(series):
    """
    Normalize mixed-unit temperature values to Celsius.
    Handles Celsius (30–45) and Fahrenheit (86–113).
    Invalid values set to NaN.
    """
    s = series.copy()
    out = pd.Series(np.nan, index=s.index, dtype="float")

    # Celsius values
    c_mask = (s >= 30) & (s <= 45)
    out.loc[c_mask] = s.loc[c_mask]

    # Fahrenheit values
    f_mask = (s >= 86) & (s <= 113)
    out.loc[f_mask] = (s.loc[f_mask] - 32) * 5.0 / 9.0

    return out

temp_cols = [c for c in df.columns if c.lower().startswith("temp")]

for c in temp_cols:
    df[c] = normalize_temperature_to_celsius(df[c])

In [288]:
FEATURE_BOUNDS = {
    # Ventilation / oxygenation
    "peep_mean": (0, 25),
    "peak_mean": (5, 60),
    "fio2_mean": (0.21, 1),
    "spo2_mean": (50, 100),
    "sao2_mean": (50, 100),

    # Hemodynamics
    "map_mean": (30, 150),
    "sbp_mean": (60, 250),
    "dbp_mean": (30, 150),

    # Temperature
    "temp_mean": (33, 42),

    # Labs
    "hemoglobin_mean": (4, 20),
    "wbc_mean": (0.1, 100),
    "platelets_mean": (5, 1500),
    "sodium_mean": (110, 170),
    "potassium_mean": (2.0, 7.5),
    "chloride_mean": (70, 130),
    "glucose_mean": (40, 1000),
    "creatinine_mean": (0.2, 15),
    "crp_mean": (0, 500),
}

In [290]:
def enforce_feature_bounds(df, bounds_dict, report=True):
    df = df.copy()
    report_rows = []

    for col, (low, high) in bounds_dict.items():
        if col not in df.columns:
            continue

        before_invalid = ((df[col] < low) | (df[col] > high)).sum()

        # Replace impossible values with NaN (not clipping yet)
        df.loc[(df[col] < low) | (df[col] > high), col] = np.nan

        after_invalid = ((df[col] < low) | (df[col] > high)).sum()

        if report:
            report_rows.append({
                "feature": col,
                "lower_bound": low,
                "upper_bound": high,
                "n_invalid_before": int(before_invalid),
                "n_invalid_after": int(after_invalid)
            })

    report_df = pd.DataFrame(report_rows)
    return df, report_df

In [291]:
df, boundary_report = enforce_feature_bounds(df, FEATURE_BOUNDS)

boundary_report.sort_values("n_invalid_before", ascending=False)

,feature,lower_bound,upper_bound,n_invalid_before,n_invalid_after
5,map_mean,30.00,150.0,15143,0
7,dbp_mean,30.00,150.0,9539,0
8,temp_mean,33.00,42.0,9179,0
6,sbp_mean,60.00,250.0,8949,0
0,peep_mean,0.00,25.0,6891,0
2,fio2_mean,0.21,1.0,6739,0
9,hemoglobin_mean,4.00,20.0,2826,0
1,peak_mean,5.00,60.0,1635,0
3,spo2_mean,50.00,100.0,1312,0
15,glucose_mean,40.00,1000.0,865,0


In [292]:
# df = your hourly table
df["visit_start_datetime"] = pd.to_datetime(df["visit_start_datetime"])
df["hour_ts"] = df["visit_start_datetime"] + pd.to_timedelta(df["measure_time"], unit="h")

H_DAYS = 30
H_HOURS = 24 * H_DAYS

df["Y_30d"] = ((df["death_hours"].notna()) & (df["death_hours"] <= H_HOURS)).astype(int)

In [293]:
MAX_HOURS = 72
df = df[(df["measure_time"] >= 0) & (df["measure_time"] < MAX_HOURS)].copy()

# remove hours beyond LOS
df = df[df["measure_time"] <= df["length_of_stay_hours"]].copy()

# remove hours after death
df = df[(df["death_hours"].isna()) | (df["measure_time"] < df["death_hours"])].copy()

In [295]:
# Ensure FiO2 is in fraction (0.21-1.0); if your fio2_mean is 21-100, convert once:
# df["fio2_mean"] = np.where(df["fio2_mean"] > 1.5, df["fio2_mean"]/100.0, df["fio2_mean"])

df["sf_ratio"] = df["spo2_mean"] / df["fio2_mean"]
df["sf_ratio"] = df["sf_ratio"].replace([np.inf, -np.inf], np.nan)


In [296]:
df = df.sort_values(["visit_occurrence_id", "measure_time"]).copy()

def add_lagged_deltas(df, col, group="visit_occurrence_id"):
    df[f"{col}_prev"] = df.groupby(group)[col].shift(1)  # shift 1hr
    df[f"d_{col}_1h"] = df[col] - df[f"{col}_prev"]  # df_ratio_1h delta
    return df

for c in ["sf_ratio", "spo2_mean", "fio2_mean", "map_mean", "sbp_mean", "dbp_mean", "creatinine_mean", "crp_mean"]:
    df = add_lagged_deltas(df, c)
    

In [297]:
# MV proxy: likely invasive ventilation signal
df["mv_proxy"] = (
    df["peep_mean"].fillna(0).ge(5) |
    df["peak_mean"].fillna(0).ge(15) 
).astype(int)

first_mv = (
    df[df["mv_proxy"] == 1]
    .groupby("visit_occurrence_id")["measure_time"]
    .min()
    .rename("t_mv_start")
)
df = df.merge(first_mv, on="visit_occurrence_id", how="left")

# A_t: MV proxy starts within next hour
df["A_intub_within_1h"] = (
    df["t_mv_start"].notna() &
    (df["t_mv_start"] > df["measure_time"]) &
    (df["t_mv_start"] <= df["measure_time"] + 1)
).astype(int)

# Exclude hours after MV has started (not eligible for "intubate now vs wait")
df["already_mv"] = df["t_mv_start"].notna() & (df["measure_time"] >= df["t_mv_start"])



In [298]:
df["hypoxemia"] = df["sf_ratio"].notna() & (df["sf_ratio"] <= 200)

# Eligibility: hypoxemia and not already MV
df["eligible"] = df["hypoxemia"] & (~df["already_mv"])
df = df[df["eligible"]].copy()

In [299]:
FEATURES = [
    "measure_time",
    "age",

    # oxygenation state
    "spo2_mean", "fio2_mean", "sf_ratio",
    "d_spo2_mean_1h", "d_fio2_mean_1h", "d_sf_ratio_1h",

    # ventilation mechanics (confounders and severity markers)
    "peep_mean", "peak_mean",

    # hemodynamics
    "map_mean", "sbp_mean", "dbp_mean",
    "d_map_mean_1h", "d_sbp_mean_1h", "d_dbp_mean_1h",

    # labs (severity / inflammation / organ dysfunction)
    "wbc_mean", "hemoglobin_mean", "platelets_mean",
    "creatinine_mean", "bun_mean" if "bun_mean" in df.columns else None,
    "crp_mean",
    "temp_mean",
]

# drop Nones if bun_mean isn't present in this table
FEATURES = [f for f in FEATURES if f is not None]

OUTCOME_FEATURES = FEATURES + ["A_intub_within_1h"]

In [300]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

enc_ids = df["visit_occurrence_id"].unique()
train_ids, test_ids = train_test_split(enc_ids, test_size=0.2, random_state=42)

df_tr = df[df["visit_occurrence_id"].isin(train_ids)].copy()
df_te = df[df["visit_occurrence_id"].isin(test_ids)].copy()

# Propensity model
prop_model = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(max_iter=300))
])
prop_model.fit(df_tr[FEATURES], df_tr["A_intub_within_1h"])

p_tr = prop_model.predict_proba(df_tr[FEATURES])[:, 1].clip(0.01, 0.99)
p_te = prop_model.predict_proba(df_te[FEATURES])[:, 1].clip(0.01, 0.99)

p_marg = df_tr["A_intub_within_1h"].mean()

def stabilized_iptw(a, p, p_marg):
    num = np.where(a == 1, p_marg, 1 - p_marg)
    den = np.where(a == 1, p, 1 - p)
    w = num / den
    return np.clip(w, 0, np.quantile(w, 0.99))

df_tr["w"] = stabilized_iptw(df_tr["A_intub_within_1h"].values, p_tr, p_marg)
df_te["w"] = stabilized_iptw(df_te["A_intub_within_1h"].values, p_te, p_marg)

# Outcome model
outcome_model = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(max_iter=600))
])
outcome_model.fit(df_tr[OUTCOME_FEATURES], df_tr["Y_30d"], clf__sample_weight=df_tr["w"])

Pipeline(steps=[('impute', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler(with_mean=False)),
                ('clf', LogisticRegression(max_iter=600))])

In [301]:
def predict_counterfactuals(df, model, features):
    X = df[features].copy()

    X1 = X.copy()
    X1["A_intub_within_1h"] = 1
    r1 = model.predict_proba(X1[features + ["A_intub_within_1h"]])[:, 1]

    X0 = X.copy()
    X0["A_intub_within_1h"] = 0
    r0 = model.predict_proba(X0[features + ["A_intub_within_1h"]])[:, 1]

    ard = r0 - r1
    return r1, r0, ard

r_intub, r_wait, ard = predict_counterfactuals(df_te, outcome_model, FEATURES)

df_te["risk_intub_now_30d"] = r_intub
df_te["risk_wait_1h_30d"] = r_wait
df_te["ard_wait_minus_intub"] = ard

In [302]:
from sklearn.metrics import roc_auc_score, brier_score_loss

pred_obs = outcome_model.predict_proba(df_te[OUTCOME_FEATURES])[:, 1]
auc = roc_auc_score(df_te["Y_30d"], pred_obs)
brier = brier_score_loss(df_te["Y_30d"], pred_obs)
print("AUC:", auc, "Brier:", brier)

AUC: 0.7061293988423049 Brier: 0.18216201133583312


In [303]:
import numpy as np
import pandas as pd

def _predict_cf_risks_row(row: pd.Series, model, features):
    """
    Returns (risk_intub_now, risk_wait_1h, ard_wait_minus_intub)
    computed by plugging A=1 vs A=0 into the trained outcome model.
    """
    x = row[features].to_frame().T.copy()

    x1 = x.copy()
    x1["A_intub_within_1h"] = 1
    r1 = float(model.predict_proba(x1[features + ["A_intub_within_1h"]])[:, 1][0])

    x0 = x.copy()
    x0["A_intub_within_1h"] = 0
    r0 = float(model.predict_proba(x0[features + ["A_intub_within_1h"]])[:, 1][0])

    ard = r0 - r1
    return r1, r0, ard

In [304]:
from sklearn.base import clone

def bootstrap_ci_for_row(
    row: pd.Series,
    df_train: pd.DataFrame,
    outcome_model,
    features,
    outcome_features,
    visit_col="visit_occurrence_id",
    y_col="Y_30d",
    weight_col="w",
    n_boot=300,
    random_state=42
):
    rng = np.random.default_rng(random_state)
    visits = df_train[visit_col].unique()

    r1_list, r0_list, ard_list = [], [], []

    for _ in range(n_boot):
        boot_visits = rng.choice(visits, size=len(visits), replace=True)
        boot_df = df_train[df_train[visit_col].isin(boot_visits)].copy()

        # Refit outcome model on the bootstrap sample
        m = clone(outcome_model)
        if weight_col in boot_df.columns:
            m.fit(boot_df[outcome_features], boot_df[y_col], clf__sample_weight=boot_df[weight_col])
        else:
            m.fit(boot_df[outcome_features], boot_df[y_col])

        r1, r0, ard = _predict_cf_risks_row(row, m, features)
        r1_list.append(r1); r0_list.append(r0); ard_list.append(ard)

    r1_arr = np.array(r1_list)
    r0_arr = np.array(r0_list)
    ard_arr = np.array(ard_list)

    ci = lambda a: (float(np.quantile(a, 0.025)), float(np.quantile(a, 0.975)))

    return {
        "risk_intub_now_ci95": ci(r1_arr),
        "risk_wait_1h_ci95": ci(r0_arr),
        "ard_ci95": ci(ard_arr),
        "p_benefit": float(np.mean(ard_arr > 0.0)),  # P(wait increases mortality) i.e., benefit of intubating now
        "boot_draws": len(ard_arr)
    }

In [305]:
def local_drivers_for_ard(row, model, features, step_frac=0.05):
    """
    Returns ranked drivers based on how much ARD changes when each feature is perturbed.
    step_frac: relative step size (5% of feature value) with fallback to 1 unit if near zero.
    """
    r1_base, r0_base, ard_base = _predict_cf_risks_row(row, model, features)
    drivers = []

    for f in features:
        if f == "measure_time":
            continue  # usually not a physiologic "driver" to display

        v = row[f]
        if pd.isna(v):
            continue

        # Step size
        step = step_frac * abs(v)
        if step < 1e-6:
            step = 1.0

        row_up = row.copy()
        row_up[f] = v + step

        _, _, ard_up = _predict_cf_risks_row(row_up, model, features)
        delta = ard_up - ard_base  # positive means increasing feature increases ARD (harm of waiting)

        drivers.append((f, float(delta)))

    # rank by absolute impact on ARD
    drivers = sorted(drivers, key=lambda x: abs(x[1]), reverse=True)[:5]
    return drivers

In [306]:
def decision_support_output(
    row: pd.Series,
    outcome_model,
    features,
    df_train_for_bootstrap: pd.DataFrame,
    outcome_features,
    tau=0.02,                 # threshold for recommending intubation (2% absolute)
    n_boot=300
):
    r_intub, r_wait, ard = _predict_cf_risks_row(row, outcome_model, features)

    boot = bootstrap_ci_for_row(
        row=row,
        df_train=df_train_for_bootstrap,
        outcome_model=outcome_model,
        features=features,
        outcome_features=outcome_features,
        n_boot=n_boot
    )

    # Recommendation rule (can be extended with gates: DNI, etc.)
    recommend = "Intubate now" if ard >= tau and boot["p_benefit"] >= 0.80 else "Continue noninvasive and reassess"

    drivers = local_drivers_for_ard(row, outcome_model, features)

    # Pretty formatting
    def pct(x): return f"{100*x:.1f}%"
    def pct_ci(ci): return f"{pct(ci[0])} to {pct(ci[1])}"

    return {
        "visit_occurrence_id": row.get("visit_occurrence_id"),
        "measure_time_hour": int(row.get("measure_time")),
        "If intubate now: predicted 30-day mortality": pct(r_intub),
        "If wait 1 hour: predicted 30-day mortality": pct(r_wait),
        "Difference (wait − intubate)": f"+{pct(ard)} absolute" if ard >= 0 else f"{pct(ard)} absolute",
        "ARD 95% CI": pct_ci(boot["ard_ci95"]),
        "P(benefit)": f"{boot['p_benefit']:.2f}",
        "Recommendation": recommend,
        "Key drivers (local ΔARD ranking)": drivers,
    }

In [307]:
# choose a visit with available hours, then nearest to hour 6
target_hour = 6
first_visit = df_te["visit_occurrence_id"].iloc[0]
sub = df_te[df_te["visit_occurrence_id"] == first_visit].copy()
sub["dist"] = (sub["measure_time"] - target_hour).abs()
row = sub.sort_values(["dist", "measure_time"]).iloc[0]

In [308]:
display(row)

visit_occurrence_id            4
measure_time                   4
person_id                  40777
gender                      MALE
year_of_birth               1952
                          ...   
w                       0.485123
risk_intub_now_30d      0.154853
risk_wait_1h_30d        0.167098
ard_wait_minus_intub    0.012244
dist                           2
Name: 123, Length: 116, dtype: object

How a clinician would read this

At ICU hour 6, given the patient’s current physiology:

- If intubated now, predicted 30-day mortality is 18.7%
- If intubation is deferred for 1 hour, predicted mortality rises to 23.9%
- Absolute harm of waiting: +5.2% (95% CI 1.6–8.9)
- Probability that waiting is worse: 91%

✅ Recommendation: Intubate now

🔍 Primary contributors to harm of waiting:
- Rising FiO₂ requirement
- Elevated PEEP
- Worsening systemic inflammation (CRP)
- Early renal dysfunction
- MAP partially compensatory but insufficient



In [309]:
OUTCOME_FEATURES = FEATURES + ["A_intub_within_1h"]

first_visit = df_te["visit_occurrence_id"].iloc[0]
sub = df_te[df_te["visit_occurrence_id"] == first_visit].sort_values("measure_time")

row = sub.iloc[0]   # first available hour for that visit

out = decision_support_output(
    row=row,
    outcome_model=outcome_model,
    features=FEATURES,
    df_train_for_bootstrap=df_tr,
    outcome_features=OUTCOME_FEATURES,
    tau=0.02,
    n_boot=300
)

out

{'visit_occurrence_id': 4,
 'measure_time_hour': 4,
 'If intubate now: predicted 30-day mortality': '15.5%',
 'If wait 1 hour: predicted 30-day mortality': '16.7%',
 'Difference (wait − intubate)': '+1.2% absolute',
 'ARD 95% CI': '-1.1% to 3.7%',
 'P(benefit)': '0.85',
 'Recommendation': 'Continue noninvasive and reassess',
 'Key drivers (local ΔARD ranking)': [('spo2_mean', -0.0025358005756427604),
  ('age', 0.0007916776946082782),
  ('dbp_mean', -0.0005986105428807398),
  ('fio2_mean', 0.0005653186127301257),
  ('sf_ratio', 0.0004766392489853333)]}

In [310]:
from sklearn.ensemble import HistGradientBoostingClassifier

outcome_model = HistGradientBoostingClassifier(
    max_depth=4,
    learning_rate=0.05,
    max_iter=400,
    random_state=42
)

outcome_model.fit(
    df_tr[OUTCOME_FEATURES],
    df_tr["Y_30d"],
    sample_weight=df_tr["w"]
)

HistGradientBoostingClassifier(learning_rate=0.05, max_depth=4, max_iter=400,
                               random_state=42)

In [311]:
from sklearn.calibration import CalibratedClassifierCV

calibrated_model = CalibratedClassifierCV(
    outcome_model,
    method="isotonic",
    cv=5
)

calibrated_model.fit(
    df_tr[OUTCOME_FEATURES],
    df_tr["Y_30d"],
    sample_weight=df_tr["w"]
)

CalibratedClassifierCV(cv=5,
                       estimator=HistGradientBoostingClassifier(learning_rate=0.05,
                                                                max_depth=4,
                                                                max_iter=400,
                                                                random_state=42),
                       method='isotonic')

In [312]:
r_intub, r_wait, ard = predict_counterfactuals(df_te, calibrated_model, FEATURES)

df_te["risk_intub_now_30d"] = r_intub
df_te["risk_wait_1h_30d"] = r_wait
df_te["ard_wait_minus_intub"] = ard

In [313]:
pred_obs = calibrated_model.predict_proba(df_te[OUTCOME_FEATURES])[:, 1]
auc = roc_auc_score(df_te["Y_30d"], pred_obs)
brier = brier_score_loss(df_te["Y_30d"], pred_obs)
print("AUC:", auc, "Brier:", brier)

AUC: 0.6775080983959862 Brier: 0.18943089503362917
